In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# src files
from situation_testing.situation_testing import SituationTesting
# plot params
plt.style.use('seaborn-whitegrid')
plt.rc('font', size=11)
plt.rc('legend', fontsize=11)
plt.rc('lines', linewidth=2)
plt.rc('axes', linewidth=2)
plt.rc('axes', edgecolor='k')
plt.rc('xtick.major', width=2)
plt.rc('xtick.major', size=6)
plt.rc('ytick.major', width=2)
plt.rc('ytick.major', size=6)
plt.rc('pdf', fonttype=42)
plt.rc('ps', fonttype=42)

In [ ]:
# working directory
wd = os.path.dirname(os.getcwd())
# relevant folders
data_path = os.path.abspath(os.path.join(wd, 'data')) + '\\'
resu_path = os.path.abspath(os.path.join(wd, 'results')) + '\\'

In [ ]:
# load and modify factual data accordingly (same for all SCFs versions)
org_df = pd.read_csv(data_path + 'clean_LawSchool.csv', sep='|').reset_index(drop=True)
print(org_df.shape)
print(org_df.columns.to_list())
org_df.head(5)

In [ ]:
# we focus on sex and race_nonwhite
df = org_df[['sex', 'race_nonwhite', 'LSAT', 'UGPA']].copy()
# df['sex'] = df['sex'].map({'Male': 0, 'Female': 1})
# df['race_nonwhite'] = df['race_nonwhite'].map({'White': 0, 'NonWhite': 1})
df.rename(columns={'sex': 'Gender', 'race_nonwhite': 'Race'}, inplace=True)
df.head(5)

### A 'known' decision maker

To frame it as a (discrete) decision making process, we include an *admissions officer* based on [the "known" requirements of US Law Schools](https://schools.lawschoolnumbers.com/). We assume the case for Yale School. We could not find a one-to-one converter between LSAT over 48 to over 180. 173/180 is 96%; it would be abloud 46.1/48. Let's assume Yale cuts at these median values, and puts a slightly higher weight on UGPA over LSAT.

In [ ]:
# Our decision maker:
b1 = 0.6
b2 = 0.4
min_score = round(b1*3.93 + b2*46.1, 2)  # 20.8
max_score = round(b1*4.00 + b2*48.00)    # 22

In [ ]:
# add the decision maker
df['Score'] = b1*df['UGPA'] + b2*df['LSAT']
df['Y'] = np.where(df['Score'] >= min_score, 1, 0)
df.head(5)

### $|\mathbf{A}=1|$: Gender

We focus on the Level 3 SCFs.

In [ ]:
do = 'Male'
org_cf_df = pd.read_csv(data_path + '\\counterfactuals\\' + f'cf_LawSchool_lev3_do{do}.csv', sep='|').reset_index(drop=True)
print(org_cf_df.shape)
print(org_cf_df.columns.to_list())
org_cf_df.head(5)

In [ ]:
cf_df = org_cf_df[['Sex', 'Race', 'scf_LSAT', 'scf_UGPA']].copy()
# cf_df['Sex'] = cf_df['Sex'].map({'Male': 0, 'Female': 1})
# cf_df['Race'] = cf_df['Race'].map({'White': 0, 'NonWhite': 1})
cf_df = cf_df.rename(columns={'Sex': 'Gender', 'scf_LSAT': 'LSAT', 'scf_UGPA': 'UGPA'})

# add the decision maker
cf_df['Score'] = b1*cf_df['UGPA'] + b2*cf_df['LSAT']
cf_df['Y'] = np.where(cf_df['Score'] >= min_score, 1, 0)
cf_df.head(5)

In [ ]:
# store do:=Male results
m_res_df = df[['Gender', 'Race', 'Y']].copy()
m_res_df['cf_Y'] = cf_df[['Y']].copy()
m_res_df.head(5)

In [ ]:
print(df[df['Gender'] == 'Female'].shape[0] / df.shape[0] * 100)
print(df[df['Race'] == 'NonWhite'].shape[0] / df.shape[0] * 100)

In [ ]:
print(df[(df['Gender'] == 'Female') & (df['Y']==1)].shape[0] / df[df['Gender'] == 'Female'].shape[0] * 100)
print(df[(df['Gender'] == 'Male') & (df['Y']==1)].shape[0] / df[df['Gender'] == 'Male'].shape[0] * 100)

print(df[(df['Race'] == 'NonWhite') & (df['Y']==1)].shape[0] / df[df['Race'] == 'NonWhite'].shape[0] * 100)
print(df[(df['Race'] == 'White') & (df['Y']==1)].shape[0] / df[df['Race'] == 'White'].shape[0] * 100)

In [ ]:
# --- attribute-specific params
feat_trgt = 'Y'
feat_trgt_vals = {'positive': 1, 'negative': 0}
# list of relevant features
feat_rlvt = ['LSAT', 'UGPA']
# protected feature
feat_prot = 'Gender'
# values for the protected feature: use 'non_protected' and 'protected' accordingly
feat_prot_vals = {'non_protected': 'Male', 'protected': 'Female'}

# --- st-specific params
# size of neiuborhoods
# n = 15
# significance level
alpha = 0.05
# tau diviation
tau = 0.0

In [ ]:
n = 30

#### Standard ST (stST)

Notice that, by construction, by not specifying 'Race' (or 'Gender') as a relevant feature, ST will ignore it.

In [ ]:
test_df = df.copy()

st = SituationTesting()
st.setup_baseline(test_df, nominal_atts=['Gender'], continuous_atts=['LSAT', 'UGPA'])

m_res_df['stST'] = st.run(target_att=feat_trgt, target_val=feat_trgt_vals,  
                          sensitive_att=feat_prot, sensitive_val=feat_prot_vals,
                          k=n, alpha=alpha, tau=tau)

In [ ]:
m_res_df[(m_res_df['stST'] > tau)].shape[0]

In [ ]:
# positive discrimination?
m_res_df[(m_res_df['stST'] < tau)].shape[0]

#### Counterfactual Situation Testing (cfST)

In [ ]:
test_df    = df.copy()
test_cf_df = cf_df.copy()

# don't include the centers
cf_st = SituationTesting()
cf_st.setup_baseline(test_df, test_cf_df, nominal_atts=['Gender'], continuous_atts=['LSAT', 'UGPA'])

m_res_df['cfST'] = cf_st.run(target_att=feat_trgt, target_val=feat_trgt_vals, 
                             sensitive_att=feat_prot, sensitive_val=feat_prot_vals,
                             include_centers=False,
                             k=n, alpha=alpha, tau=tau)

In [ ]:
m_res_df[(m_res_df['cfST'] > tau)].shape[0]

In [ ]:
# positive discrimination?
m_res_df[(m_res_df['cfST'] < tau)].shape[0]

#### Counterfactual Fairness

In [ ]:
m_res_df['CF'] = cf_st.res_counterfactual_unfairness

In [ ]:
m_res_df[m_res_df['CF'] == 1].shape[0]

In [ ]:
# positive discrimination?
m_res_df[m_res_df['CF'] == 2].shape[0] 

#### cfST with centers

In [ ]:
test_df    = df.copy()
test_cf_df = cf_df.copy()

# don't include the centers
wcf_st = SituationTesting()
wcf_st.setup_baseline(test_df, test_cf_df, nominal_atts=['Gender'], continuous_atts=['LSAT', 'UGPA'])

m_res_df['cfST_w'] = wcf_st.run(target_att=feat_trgt, target_val=feat_trgt_vals, 
                                sensitive_att=feat_prot, sensitive_val=feat_prot_vals,
                                include_centers=True,
                                k=n, alpha=alpha, tau=tau)

In [ ]:
m_res_df[(m_res_df['cfST_w'] > tau)].shape[0]

In [ ]:
# positive discrimination?
m_res_df[(m_res_df['cfST_w'] < tau)].shape[0]

### Add CIs for CF

In [ ]:
df_wcf_sf_disc = wcf_st.get_test_discrimination()
df_wcf_sf_disc.head(5)

In [ ]:
# add this to results to showcase the method
df_wcf_sf_disc[df_wcf_sf_disc['individual'].isin(m_res_df[m_res_df['CF'] == True].index.to_list())].head(5)

### $|\mathbf{A}=1|$: Race

We focus on the Level 3 SCFs.

In [ ]:
do = 'White'
org_cf_df = pd.read_csv(data_path + '\\counterfactuals\\' + f'cf_LawSchool_lev3_do{do}.csv', sep='|').reset_index(drop=True)
print(org_cf_df.shape)
print(org_cf_df.columns.to_list())
org_cf_df.head(5)

In [ ]:
cf_df = org_cf_df[['Sex', 'Race', 'scf_LSAT', 'scf_UGPA']].copy()
cf_df = cf_df.rename(columns={'Sex': 'Gender', 'scf_LSAT': 'LSAT', 'scf_UGPA': 'UGPA'})

# add the decision maker
cf_df['Score'] = b1*cf_df['UGPA'] + b2*cf_df['LSAT']
cf_df['Y'] = np.where(cf_df['Score'] >= min_score, 1, 0)
cf_df.head(5)

In [ ]:
# df.groupby(['Y', 'Race']).count()
# cf_df.groupby(['Y', 'Race']).count()

In [ ]:
# store do:=White results
w_res_df = df[['Gender', 'Race', 'Y']].copy()
w_res_df['cf_Y'] = cf_df[['Y']].copy()
w_res_df.head(5)

In [ ]:
# attribute-specific params
feat_trgt = 'Y'
feat_trgt_vals = {'positive': 1, 'negative': 0}
# list of relevant features
feat_rlvt = ['LSAT', 'UGPA']
# protected feature
feat_prot = 'Race'
# values for the protected feature: use 'non_protected' and 'protected' accordingly
feat_prot_vals = {'non_protected': 'White', 'protected': 'NonWhite'}

# st-specific params
# size of neiuborhoods
# n = 15
# significance level
alpha = 0.05
# tau diviation
tau = 0.0

In [ ]:
n = 30

#### Standard ST (stST)

Notice that, by construction, by not specifying 'Race' (or 'Gender') as a relevant feature, ST will ignore it.

In [ ]:
test_df = df.copy()

st = SituationTesting()
st.setup_baseline(test_df, nominal_atts=['Race'], continuous_atts=['LSAT', 'UGPA'])

w_res_df['stST'] = st.run(target_att=feat_trgt, target_val=feat_trgt_vals,  
                          sensitive_att=feat_prot, sensitive_val=feat_prot_vals,
                          k=n, alpha=alpha, tau=tau)

In [ ]:
w_res_df[(w_res_df['stST'] > tau)].shape[0]

In [ ]:
# positive discrimination?
w_res_df[(w_res_df['stST'] < tau)].shape[0]

#### Counterfactual Situation Testing (cfST)

In [ ]:
test_df    = df.copy()
test_cf_df = cf_df.copy()

# don't include the centers
cf_st = SituationTesting()
cf_st.setup_baseline(test_df, test_cf_df, nominal_atts=['Race'], continuous_atts=['LSAT', 'UGPA'])

w_res_df['cfST'] = cf_st.run(target_att=feat_trgt, target_val=feat_trgt_vals, 
                             sensitive_att=feat_prot, sensitive_val=feat_prot_vals,
                             include_centers=False,
                             k=n, alpha=alpha, tau=tau)

In [ ]:
w_res_df[(w_res_df['cfST'] > tau)].shape[0]

In [ ]:
# positive discrimination?
w_res_df[(w_res_df['cfST'] < tau)].shape[0]

#### Counterfactual Fairness

In [ ]:
w_res_df['CF'] = cf_st.res_counterfactual_unfairness

In [ ]:
w_res_df[w_res_df['CF'] == 1].shape[0]

In [ ]:
w_res_df[w_res_df['CF'] == 2].shape[0]

#### cfST with centers

In [ ]:
test_df    = df.copy()
test_cf_df = cf_df.copy()

# include the centers
wcf_st = SituationTesting()
wcf_st.setup_baseline(test_df, test_cf_df, nominal_atts=['Race'], continuous_atts=['LSAT', 'UGPA'])

w_res_df['cfST_w'] = wcf_st.run(target_att=feat_trgt, target_val=feat_trgt_vals, 
                                sensitive_att=feat_prot, sensitive_val=feat_prot_vals,
                                include_centers=True,
                                k=n, alpha=alpha, tau=tau)

In [ ]:
w_res_df[(w_res_df['cfST_w'] > tau)].shape[0]

In [ ]:
# positive discrimination?
w_res_df[(w_res_df['cfST_w'] < tau)].shape[0]

### Multiple discrimination: $|\mathbf{A}|=2$

In [ ]:
df[(df['Gender'] == 'Female') & (df['Race'] == 'NonWhite')].shape[0]

In [ ]:
# for stST
pd.merge(left=m_res_df[m_res_df['stST'] > tau], right=w_res_df[w_res_df['stST'] > tau], 
         how='inner', 
         left_index=True, right_index=True).shape[0]

In [ ]:
# for stST +
pd.merge(left=m_res_df[m_res_df['stST'] < tau], right=w_res_df[w_res_df['stST'] < tau], 
         how='inner', 
         left_index=True, right_index=True).shape[0]

In [ ]:
# for cfST
pd.merge(left=m_res_df[m_res_df['cfST'] > tau], right=w_res_df[w_res_df['cfST'] > tau], 
         how='inner', 
         left_index=True, right_index=True).shape[0]

In [ ]:
# for cfST +
pd.merge(left=m_res_df[m_res_df['cfST'] < tau], right=w_res_df[w_res_df['cfST'] < tau], 
         how='inner', 
         left_index=True, right_index=True).shape[0]

In [ ]:
# for stST_w
pd.merge(left=m_res_df[m_res_df['cfST_w'] > tau], right=w_res_df[w_res_df['cfST_w'] > tau], 
         how='inner', 
         left_index=True, right_index=True).shape[0]

In [ ]:
# for stST_w +
pd.merge(left=m_res_df[m_res_df['cfST_w'] < tau], right=w_res_df[w_res_df['cfST_w'] < tau], 
         how='inner', 
         left_index=True, right_index=True).shape[0]

In [ ]:
# for Counterfactual Fairness
pd.merge(left=m_res_df[m_res_df['CF'] == 1], right=w_res_df[w_res_df['CF'] == 1], 
         how='inner', 
         left_index=True, right_index=True).shape[0]

In [ ]:
# for Counterfactual Fairness w +
pd.merge(left=m_res_df[m_res_df['CF'] == 2], right=w_res_df[w_res_df['CF'] == 2], 
         how='inner', 
         left_index=True, right_index=True).shape[0]

### Intersectional Discrimination: $A_1 \cap A_2$

In [ ]:
do = 'MaleWhite'
org_cf_df = pd.read_csv(data_path + '\\counterfactuals\\' + f'cf_LawSchool_lev3_do{do}.csv', sep='|').reset_index(drop=True)
print(org_cf_df.shape)
print(org_cf_df.columns.to_list())
org_cf_df.head(5)

In [ ]:
cf_df = org_cf_df[['GenderRace', 'scf_LSAT', 'scf_UGPA']].copy()
cf_df = cf_df.rename(columns={'scf_LSAT': 'LSAT', 'scf_UGPA': 'UGPA'})

# add the decision maker
cf_df['Score'] = b1*cf_df['UGPA'] + b2*cf_df['LSAT']
cf_df['Y'] = np.where(cf_df['Score'] >= min_score, 1, 0)
cf_df.head(5)

In [ ]:
# add the intersectional var to df
df.head()
df['GenderRace'] =  df['Gender'] + '-' + df['Race']
df.head(5)

In [ ]:
# df.groupby(['Y', 'GenderRace']).count()
# cf_df.groupby(['Y', 'GenderRace']).count()

In [ ]:
# store do:=White results
int_res_df = df[['Gender', 'Race', 'Y']].copy()
int_res_df['cf_Y'] = cf_df[['Y']].copy()
int_res_df.head(5)

In [ ]:
# attribute-specific params
feat_trgt = 'Y'
feat_trgt_vals = {'positive': 1, 'negative': 0}
# list of relevant features
feat_rlvt = ['LSAT', 'UGPA']
# protected feature
feat_prot = 'GenderRace'
# values for the protected feature: use 'non_protected' and 'protected' accordingly
feat_prot_vals = {'non_protected': ['Female-White', 'Male-NonWhite', 'Male-NonWhite', 'Male-White'], 
                  'protected': 'Female-NonWhite'
                 }

# st-specific params
# size of neiuborhoods
n = 15
# significance level
alpha = 0.05
# tau diviation
tau = 0.0

#### Standard ST (stST)

In [ ]:
test_df = df.copy()

st = SituationTesting()
st.setup_baseline(test_df, nominal_atts=['GenderRace'], continuous_atts=['LSAT', 'UGPA'])

int_res_df['stST'] = st.run(target_att=feat_trgt, target_val=feat_trgt_vals,
                            sensitive_att=feat_prot, sensitive_val=feat_prot_vals,
                            k=n, alpha=alpha, tau=tau)

In [ ]:
int_res_df[(int_res_df['stST'] > tau)].shape[0]

In [ ]:
# positive discrimination?
int_res_df[(int_res_df['stST'] < tau)].shape[0]

#### Counterfactual Situation Testing (cfST)

In [ ]:
test_df    = df.copy()
test_cf_df = cf_df.copy()

# don't include the centers
cf_st = SituationTesting()
cf_st.setup_baseline(test_df, test_cf_df, nominal_atts=['Race'], continuous_atts=['LSAT', 'UGPA'])

int_res_df['cfST'] = cf_st.run(target_att=feat_trgt, target_val=feat_trgt_vals, 
                               sensitive_att=feat_prot, sensitive_val=feat_prot_vals,
                               include_centers=False,
                               k=n, alpha=alpha, tau=tau)

In [ ]:
int_res_df[(int_res_df['cfST'] > tau)].shape[0]

In [ ]:
# positive discrimination?
int_res_df[(int_res_df['cfST'] < tau)].shape[0]

#### Counterfactual Fairness

In [ ]:
int_res_df['CF'] = cf_st.res_counterfactual_unfairness

In [ ]:
int_res_df[int_res_df['CF'] == 1].shape[0]

In [ ]:
int_res_df[int_res_df['CF'] == 2].shape[0]

#### cfST with centers

In [ ]:
test_df    = df.copy()
test_cf_df = cf_df.copy()

# include the centers
wcf_st = SituationTesting()
wcf_st.setup_baseline(test_df, test_cf_df, nominal_atts=['Race'], continuous_atts=['LSAT', 'UGPA'])

int_res_df['cfST_w'] = wcf_st.run(target_att=feat_trgt, target_val=feat_trgt_vals, 
                                  sensitive_att=feat_prot, sensitive_val=feat_prot_vals,
                                  include_centers=True,
                                  k=n, alpha=alpha, tau=tau)

In [ ]:
int_res_df[(int_res_df['cfST_w'] > tau)].shape[0]

In [ ]:
# positive discrimination?
int_res_df[(int_res_df['cfST_w'] < tau)].shape[0]